In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler 
from sklearn.preprocessing import LabelEncoder

### Load data

In [2]:
train_data = pd.read_csv('../../airbnb_project_data/train_users_2.csv')
train_data.head(10)

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
1,820tgsjxq7,2011-05-25,20090523174809,NaN,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
3,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US
5,osr2jwljor,2010-01-01,20100101215619,2010-01-02,-unknown-,NaN,basic,0,en,other,other,omg,Web,Mac Desktop,Chrome,US
6,lsw9q7uk0j,2010-01-02,20100102012558,2010-01-05,FEMALE,46.0,basic,0,en,other,craigslist,untracked,Web,Mac Desktop,Safari,US
7,0d01nltbrs,2010-01-03,20100103191905,2010-01-13,FEMALE,47.0,basic,0,en,direct,direct,omg,Web,Mac Desktop,Safari,US
8,a1vcnhxeij,2010-01-04,20100104004211,2010-07-29,FEMALE,50.0,basic,0,en,other,craigslist,untracked,Web,Mac Desktop,Safari,US
9,6uh8zyj2gn,2010-01-04,20100104023758,2010-01-04,-unknown-,46.0,basic,0,en,other,craigslist,omg,Web,Mac Desktop,Firefox,US


### Rearrange data

In [3]:
def rearrange_data(dataset):
    #rearrange age data and separate as four groups: nan, young people, middle age people and old people
    age = []
    for i in dataset.age:
        if (i > 95) | (i < 18) | (pd.isnull(i)) :
            category = 'NaN'
        elif (i >= 18) & (i < 30):
            category = 'young_people'
        elif (i >= 30) & (i < 55):
            category = 'middle_age_people'
        elif (i >= 55) & (i <= 95):
            category = 'old_people'
        age.append(category)
    age = pd.get_dummies(pd.Series(age), prefix='age', drop_first=True)
    
    #use dummy variable to separate different categories in each column
    signup_method = pd.get_dummies(dataset.signup_method, prefix='signup_method', drop_first=True)
    language = pd.get_dummies(dataset.language, prefix='language', drop_first=True) #(train.language == 'en').astype(int)
    affiliate_channel = pd.get_dummies(dataset.affiliate_channel, prefix='affiliate_channel', drop_first=True)
    first_tracked = pd.get_dummies(dataset.first_affiliate_tracked, prefix='first_tracked', drop_first=True)
    signup_app = pd.get_dummies(dataset.signup_app, prefix='signup_app', drop_first=True)
    signup_flow = pd.get_dummies(dataset.signup_flow, prefix='signup_flow', drop_first=True)
    affiliate_provider = pd.get_dummies(dataset.affiliate_provider, prefix='affiliate_provider', drop_first=True)
    first_browser = pd.get_dummies(dataset.first_browser, prefix='first_browser', drop_first=True)
    
    #for gender column, combine 'unknown' with 'other'
    dataset.gender.replace('-unknown-', 'OTHER', inplace=True)
    gender = pd.get_dummies(dataset.gender, prefix='gender', drop_first=True)
    
    #for first device type, combine 'desktop(other)', 'smartphone(other)' and 'other/unknown' as one group
    first_device = dataset.first_device_type
    pd.options.mode.chained_assignment = None
    first_device[(first_device == 'Desktop (Other)')|(first_device == 'SmartPhone (Other)')] = 'Other/Unknown'
    first_device = pd.get_dummies(first_device, prefix='first_device', drop_first=True)

    return gender, age, signup_method, signup_flow, language, affiliate_channel, affiliate_provider, first_tracked, signup_app, first_device, first_browser

### First model

In [4]:
#deal with time
#create_time = []
#for i in train_data.date_account_created:
#    create_time.append(pd.Timestamp(i))
    
#active_time = []
#for i in train_data.timestamp_first_active:
#    active_time.append(pd.Timestamp(str(i)[0:8]))
    
#gap = []
#for i in range(len(create_time)):
#    gap_day = (create_time[i] - active_time[i]).days
#    gap.append(float(gap_day))
#gap = pd.Series(gap).rename('gap')

In [5]:
gender, age, signup_method, signup_flow, language, affiliate_channel, affiliate_provider, first_tracked, signup_app, first_device, first_browser = rearrange_data(train_data)

In [6]:
first_model_label = (train_data.country_destination == 'NDF').astype(int)

In [7]:
data_df_1 = pd.concat([gender, age, signup_method, signup_flow, 
                       language, affiliate_channel, affiliate_provider, first_tracked, signup_app, first_device,
                       first_browser, first_model_label], axis=1)

In [8]:
data_df_1.head()

,gender_MALE,gender_OTHER,age_middle_age_people,age_old_people,age_young_people,signup_method_facebook,signup_method_google,signup_flow_1,signup_flow_2,signup_flow_3,...,first_browser_Silk,first_browser_SiteKiosk,first_browser_SlimBrowser,first_browser_Sogou Explorer,first_browser_Stainless,first_browser_TenFourFox,first_browser_TheWorld Browser,first_browser_Yandex.Browser,first_browser_wOSBrowser,country_destination
0,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
data_df_1.shape

(213451, 138)

In [10]:
# Splitting the data to 70% training, 20% validating and 10% testing
np.random.seed(101)
data = np.random.permutation(data_df_1)
train, validate, test = np.split(data, [int(.7*len(data)), int(.9*len(data))])

X_train = train[:, :-1]
Y_train = train[:, -1]
X_test = validate[:, :-1]
Y_test = validate[:, -1]
X_final = test[:, :-1]
Y_final = test[:, -1]

print(X_train.shape)
print(X_test.shape)
print(X_final.shape)
print(Y_train.shape)
print(Y_test.shape)
print(Y_final.shape)

(149415, 137)
(42690, 137)
(21346, 137)
(149415,)
(42690,)
(21346,)


In [11]:
print(sum(Y_train)/len(Y_train), sum(Y_test)/len(Y_test))

0.582485024931 0.584680252987


In [12]:
# Standardize variables
#scaler = StandardScaler()
#scaler.fit(X_train) # use train data to fit
#X_test = scaler.transform(X_test)
#X_final = scaler.transform(X_final)
#X_train = scaler.transform(X_train)
#X_train

In [14]:
model1 = LinearSVC(class_weight='balanced')
model1.fit(X_train, Y_train)

LinearSVC(C=1.0, class_weight='balanced', dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [20]:
# Try gridsearch for tuning parameters
parameters = {'C':[10, 20, 30], 'tol':[1e-06, 1e-05, 1e-04]}
clf = GridSearchCV(model1, parameters, cv=5, scoring='accuracy')
clf.fit(X_train, Y_train)
clf.best_estimator_, clf.best_params_, clf.cv_results_['params'], clf.cv_results_['mean_test_score']

(LinearSVC(C=20, class_weight='balanced', dual=True, fit_intercept=True,
      intercept_scaling=1, loss='squared_hinge', max_iter=1000,
      multi_class='ovr', penalty='l2', random_state=None, tol=1e-05,
      verbose=0),
 {'C': 20, 'tol': 1e-05},
 ({'C': 10, 'tol': 1e-06},
  {'C': 10, 'tol': 1e-05},
  {'C': 10, 'tol': 0.0001},
  {'C': 20, 'tol': 1e-06},
  {'C': 20, 'tol': 1e-05},
  {'C': 20, 'tol': 0.0001},
  {'C': 30, 'tol': 1e-06},
  {'C': 30, 'tol': 1e-05},
  {'C': 30, 'tol': 0.0001}),
 array([ 0.68350567,  0.68386708,  0.6830171 ,  0.68091557,  0.6865375 ,
         0.68378008,  0.67638457,  0.66803199,  0.68400094]))

In [18]:
model = LinearSVC(C=20, tol=1e-05, class_weight='balanced')
model.fit(X_train, Y_train)

LinearSVC(C=20, class_weight='balanced', dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=1e-05,
     verbose=0)

In [19]:
print(classification_report(y_true=Y_test, y_pred=model.predict(X_test), target_names=['no', 'yes']))

             precision    recall  f1-score   support

         no       0.63      0.63      0.63     17730
        yes       0.73      0.73      0.73     24960

avg / total       0.69      0.69      0.69     42690



### Second model

In [35]:
had_first_book = train_data[train_data.country_destination != 'NDF']
had_first_book.reset_index(drop=True, inplace=True)
had_first_book.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
1,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
2,87mebub9p4,2010-09-14,20091208061105,2010-02-18,OTHER,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US
3,osr2jwljor,2010-01-01,20100101215619,2010-01-02,OTHER,NaN,basic,0,en,other,other,omg,Web,Mac Desktop,Chrome,US
4,lsw9q7uk0j,2010-01-02,20100102012558,2010-01-05,FEMALE,46.0,basic,0,en,other,craigslist,untracked,Web,Mac Desktop,Safari,US


In [36]:
# deal with time
#booking_year = []
#booking_month = []
#for i in had_first_book.date_first_booking:
#    y, m, d = i.split('-')
#    booking_year.append(y)
#    booking_month.append(m)
    
#first_booking_y = pd.get_dummies(pd.Series(booking_year), prefix='first_booking_y', drop_first=True)
#first_booking_m = pd.get_dummies(pd.Series(booking_month), prefix='first_booking_m', drop_first=True)

In [37]:
gender, age, signup_method, signup_flow, language, affiliate_channel, affiliate_provider, first_tracked, signup_app, first_device, first_browser = rearrange_data(had_first_book)

In [38]:
le = LabelEncoder()
le.fit(had_first_book.country_destination.unique())
second_model_label = pd.Series(le.transform(had_first_book.country_destination))

In [39]:
data_df_2 = pd.concat([gender, age, signup_method, signup_flow, 
                       language, affiliate_channel, affiliate_provider, first_tracked, signup_app, first_device,
                       first_browser, second_model_label], axis=1)

In [40]:
data_df_2.shape

(88908, 122)

In [41]:
# Splitting the data to 70% training, 20% validating and 10% testing
np.random.seed(101)
data = np.random.permutation(data_df_2)
train, validate, test = np.split(data, [int(.7*len(data)), int(.9*len(data))])

X_train = train[:, :-1]
Y_train = train[:, -1]
X_test = validate[:, :-1]
Y_test = validate[:, -1]
X_final = test[:, :-1]
Y_final = test[:, -1]

print(X_train.shape)
print(X_test.shape)
print(X_final.shape)
print(Y_train.shape)
print(Y_test.shape)
print(Y_final.shape)

(62235, 121)
(17782, 121)
(8891, 121)
(62235,)
(17782,)
(8891,)


In [43]:
# Standardize variables
#scaler = StandardScaler()
#scaler.fit(X_train) # use train data to fit
#X_test = scaler.transform(X_test)
#X_final = scaler.transform(X_final)
#X_train = scaler.transform(X_train)
#X_train

In [54]:
model2 = LinearSVC(class_weight='balanced')
model2.fit(X_train, Y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [52]:
list(le.classes_)

['AU', 'CA', 'DE', 'ES', 'FR', 'GB', 'IT', 'NL', 'PT', 'US', 'other']

In [55]:
target_names = list(le.classes_)
print(classification_report(y_true=Y_test, y_pred=model2.predict(X_test), target_names=target_names))

             precision    recall  f1-score   support

         AU       0.00      0.00      0.00       107
         CA       0.00      0.00      0.00       260
         DE       0.00      0.00      0.00       217
         ES       0.10      0.00      0.01       429
         FR       0.00      0.00      0.00      1050
         GB       0.00      0.00      0.00       470
         IT       0.00      0.00      0.00       551
         NL       0.00      0.00      0.00       156
         PT       0.00      0.00      0.00        35
         US       0.70      0.99      0.82     12452
      other       0.14      0.00      0.01      2055

avg / total       0.51      0.70      0.58     17782



/Users/yatongge/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [29]:
#pd.set_option('display.max_columns', None)